# Scene Tracing

A single detector in a homogenous medium is rarely an accurate model of an
experiment. Scenes allow to introduce arbitrary boundaries between media, such
as regions of varying scattering or the glass housing of the detector. This
notebook will show how to create them.

_We assume you already know the basics of tracing pipelines shown in the first
notebook._

In [ ]:
# Uncomment this, if you run this on Google Colab
# !sudo apt-get install -y libnvidia-gl-550 vulkan-tools
# !pip install git+https://github.com/tkerscher/theia
# !git clone --depth 1 git+https://github.com/tkerscher/theia
# !cd theia/notebooks

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import hephaistos as hp
import hephaistos.pipeline as pl

import theia
import theia.units as u

# print device we run this on
print(hp.getCurrentDevice())

## Meshes

Volume boundaries are defined by meshes consisting of triangles. To make them
reusable we place them not directly in the scene, but define them in their own
coordinate system we refer to as the _object space_. To place them in the scene,
we reference them while assigning an (affine) transformation converting the
object space to the scene's coordinate system, called the _world space_.

Once placed in the scene, the mesh partitions the volume into an _inside_
volume and an _outside_ volume using its surface normal, which, by definition,
point from the inside to the outside. Both volumes may be further partitioned
by other meshes, including placing them inside each other. This implies, that
whether a volume is inside or outside can only be answered with respect to a
specific mesh.

Loading meshes from files and later onto the GPU is handled by `MeshStore`.
It expects a dictionary mapping strings of mesh names to either a file path
or to a `hephaistos.Mesh` instance containing the mesh. Let's start with that.

In [ ]:
from theia.scene import MeshStore

meshStore = MeshStore({
    "cone": "../assets/cone.stl",
    "cube": "../assets/cube.ply",
    "sphere": "../assets/sphere.stl",
    "suzanne": "../assets/suzanne.stl",
    "torus": "../assets/torus.stl",
})

## Material

_Materials_ in combination with a specific mesh assign media to the respective
inside and outside volumes. For instance, take an ice cube in water. The
mesh would define the boundary or surface of the cube with the surface normal
pointing from the center away. The material then assigns the inside to the glass
medium and the outside to the water medium.

Note that unlike in many other 3D software, materials define both inside and
outside medium. This especially means, that if parts of your mesh touch different
media, you have to split it. Imagine we introduce in our previous example a
second cube containing a different medium. If this cube touches our ice cube,
we have to split the original mesh into the part the touches the water and the
part that touches the other cube, and assign appropriate materials to them.
Another example is shown in the following illustration.

![material illustration](../docs/images/volume_interface.svg)

Materials also allow to guide or constrain the tracer when encountering a mesh
by e.g. absorbing the ray, ignoring transmissions or creating hits.
Such behavior is defined by `flagsInwards` for light encountering the mesh from
the outside medium and `flagsOutwards` for light from the inside medium.
The available flags are:

- `BLACK_BODY`: Rays are absorbed stopping tracing immediately.
- `DETECTOR`: Indicates hits may produce a response.
- `NO_REFLECT`: Disables reflection at this boundary.
- `NO_TRANSMIT`: Disables transmission at this boundary.
- `VOLUME_BORDER`: Only transmission without refracting the ray's direction.

After this lengthy explanation we go back to coding.
Before we can create a material we need to define the media it references.
Since we already explained in the previous notebook media, we go straight to work:

In [ ]:
from theia.material import BK7Model, HenyeyGreensteinPhaseFunction, MediumModel, WaterBaseModel

# we will define two media: water and glass
class WaterModel(WaterBaseModel, HenyeyGreensteinPhaseFunction, MediumModel):
    def __init__(self):
        WaterBaseModel.__init__(
            self,
            temperature=4.0, # °C
            pressure=10_000, # dbar ~ 10cm water depth
            salinity=35.0    # PSU
        )
        HenyeyGreensteinPhaseFunction.__init__(self, 0.9)

lamMin, lamMax = 400.0 * u.nm, 800.0 * u.nm
water = WaterModel().createMedium(lamMin, lamMax, name="water")
glass = BK7Model().createMedium(lamMin, lamMax, name="glass")

Next we can define our materials. `Material` first takes a name which will be
used to reference it when building the scene, followed by the medium inside and
outside. Media can both be referenced by either passing the python object or by
its name. In the latter case we have to ensure that the corresponding python
object is either referenced by another material or pass it in a list to
`MaterialStore`. The last argument specifies the flag, where you can use a
shorthand string notation as described in the documentation of
`parseMaterialFlags`.

In [ ]:
from theia.material import Material

# inside glass, outside water
# allow transmission in both sides, but reflection only from the water side
glass_water = Material("glass_water", glass, water, flags=("TR", "T"))

# inside air, outside glass
# None denotes vacuum, which is close enough to air for us
# if we want to apply the same flags to both directions we can skip the tuple
air_glass = Material("air_glass", None, "glass", flags="T")

# detector material inside glass
# since we will mark it as absorber, the exact material won't matter
# -> choose vacuum (None)
det_water = Material("det_water", None, water, flags="BD")

Finally, to make the material available on the GPU, we have to pass everything
to `MaterialStore`:

In [ ]:
from theia.material import MaterialStore

# MaterialStore automatically collects all referenced media
# You only have to pass them if all materials reference them only by name
matStore = MaterialStore([glass_water, air_glass, det_water])

## Scene

Now that we have all the building blocks required, we can finally create a
scene. To keep things simple, we will only add a black body detector sphere and
a hollow glass sphere, i.e. an outer and an inner shell, filled with air. Later,
we will place a point light source in its center.

To actually use a Mesh, we ask `MeshStore` to create a new instance of the
mesh referenced by its name. In this step we also specify the material we want
to apply to the instance as well as the transformation translating the mesh's
triangles defined in its object space to the global shared world space.

Meshes corresponding to detectors should specify `detectorId`. The scene tracer
has a similar property. If a light ray hits a mesh and that direction has the
`DETCTOR` flag set, the tracer will check if both ids match and only then create
a hit. This is useful to select a single detector if your scene contains
multiple ones. Note that multiple meshes may share the same id.

Once all instances are created, you collect them into a list and pass it to the
new `Scene` at creation alongside the dictionary of materials provided by
`MaterialStore` used to resolve the assigned materials. Scenes also define a
tracing boundary box limiting the spread of light rays by terminating rays that
leave the box. By default it allows 1km in each primal direction from the origin.
Finally, it is good practice to define the medium of the outermost volume.

In [ ]:
from theia.scene import Scene, Transform

# sphere sizes
radius_outer = 80.0 * u.cm
radius_inner = 75.0 * u.cm
radius_det = 60.0 * u.cm
# sphere position
light_pos = (3.0, 0.0, 0.0) * u.m
det_pos = (0.0, 3.0, 0.0) * u.m

# place all three spheres
outer_sphere = meshStore.createInstance("sphere", "glass_water", Transform.TRS(scale=radius_outer, translate=light_pos))
inner_sphere = meshStore.createInstance("sphere", "air_glass", Transform.TRS(scale=radius_inner, translate=light_pos))
det_sphere = meshStore.createInstance("sphere", "det_water", Transform.TRS(scale=radius_det, translate=det_pos), detectorId=1)

# collect them in a scene
scene = Scene(
    [outer_sphere, inner_sphere, det_sphere],
    materials=matStore.material,
    medium=matStore.media["water"],    
)

## Simulation Pipeline

Creating the simulation pipeline is similar to what has been shown in the
previous notebook with the key difference being that we replace
`VolumeForwardTracer` with `SceneForwardTracer`, and `Target` with `TargetGuide`.
The latter is optionally, but omitting it causes the tracing process to create
hits only by chance, i.e. it won't create alternative light paths by sampling
direction aimed at the detector/target.

In [ ]:
from theia.light import UniformWavelengthSource, SphericalLightSource
from theia.random import PhiloxRNG
from theia.response import HistogramHitResponse, UniformValueResponse
from theia.target import SphereTargetGuide
from theia.trace import SceneForwardTracer

batch_size = 512 * 1024 # how many light paths to sample per run
sample_coef = 0.05 / u.m # "scatter" length used for sampling path segment lengths
n_scatter = 10 # Number of scatter events to simulate for each path

n_bins = 100
bin_size = 5.0 * u.ns

wavelength = UniformWavelengthSource(lambdaRange=(lamMin, lamMax))
light = SphericalLightSource(
    position=light_pos,
    timeRange=(0.0, 10.0) * u.ns,
    budget=1.0e5,
)
guide = SphereTargetGuide(position=det_pos, radius=radius_det)
response = HistogramHitResponse(
    UniformValueResponse(),
    nBins=n_bins,
    binSize=bin_size,
)
rng = PhiloxRNG(key=42)

tracer = SceneForwardTracer(
    batch_size,
    light,
    wavelength,
    response,
    rng,
    scene,
    maxPathLength=n_scatter,
    sourceMedium=0, # we start in air
    scatterCoefficient=sample_coef,
    targetId=1, # matches the detectorId
    targetGuide=guide,
)

rng.autoAdvance = tracer.nRNGSamples

Before continuing, let's look into some differences with `SceneForwardTracer`:

- `nScattering` became `maxPathLength`. The reason is that besides volume
  scattering, mesh intersection also counts towards that limit.
- `sourceMedium` specifying the medium surrounding the light became optional.
  If not specified the scene's medium will be used.
- With `targetIdx` we can "enable" a specify detector mesh or group thereof as
  explained before.

Everything hereafter should be familiar: We create the pipeline, submit some
work and plot the result.

## Finishing the Pipeline

In [ ]:
n_runs = 20

hists = []
def process(config: int, i: int) -> None:
    hists.append(response.result(config).copy())

pipeline = pl.Pipeline(tracer.collectStages())
scheduler = pl.PipelineScheduler(pipeline, processFn=process)

tasks = [{}, ] * n_runs
scheduler.schedule(tasks)
scheduler.wait()

In [ ]:
final_hist = np.mean(hists, 0)
t = np.arange(n_bins) * bin_size

plt.figure()
plt.step(t, final_hist)
plt.yscale("log")
plt.show()